# Federated MNIST Experiment

In [1]:
#Install dependencies if not already installed
!pip install torch mnist

You should consider upgrading via the '/home/pfoley1/fl_venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import mnist
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import fledge.native as fx
from fledge.federated import FederatedModel,FederatedDataSet


After importing the required packages, the next step is setting up our fledge workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── plan.yaml
│   ├── cols.yaml
│   ├── data.yaml
│   ├── plans
│   │   └── default
│   └── defaults
│       ├── collaborator.yaml
│       ├── tasks_torch.yaml
│       ├── tasks_fast_estimator.yaml
│       ├── tasks_keras.yaml
│       ├── assigner.yaml
│       ├── task_runner.yaml
│       ├── tasks_tensorflow.yaml
│       ├── network.yaml
│       ├── aggregator.yaml
│       ├── defaults
│       └── data_loader.yaml
├── data
│   ├── MNIST
│   │   ├── processed
│   │   └── raw
│   ├── cifar-10-batches-py
│   │   ├── data_batch_1
│   │   ├── test_batch
│   │   ├── data_batch_3
│   │   ├── batches.meta
│   │   ├── data_batch_2
│   │   ├── data_batch_5
│   │   ├── data_batch_4
│   │   └── readme.html
│   └── cifar-10-python.tar.gz
├── save
│   ├── last.pbuf
│   ├── init.pbuf
│   ├── torch_cnn_mnist_last.pbuf
│   ├── torch_cnn_mnist_best.pbu

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

#train_images,train_labels = trainset.train_data, trainset.train_labels
train_images,train_labels = trainset.data, np.array(trainset.targets)
train_images = torch.from_numpy(np.transpose(train_images, (0,3,1,2))).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#valid_images,valid_labels = validset.test_data, validset.test_labels
valid_images,valid_labels = validset.data, np.array(validset.targets)
#valid_images.permute(0,3,1,2)
valid_images = torch.from_numpy(np.transpose(valid_images, (0,3,1,2))).float()
valid_labels = one_hot(valid_labels,10)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
#Import training and validation images/labels
#train_images = mnist.train_images()
#train_labels = to_categorical(mnist.train_labels())
#valid_images = mnist.test_images()
#valid_labels = to_categorical(mnist.test_labels())

#def preprocess(images):
#    #Normalize
#    images = (images / 255) - 0.5
#    #Flatten
#    images = images.reshape((-1, 784))
#    return images

# Preprocess the images.
#train_images = preprocess(train_images)
#valid_images = preprocess(valid_images)

feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

net = Net()
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.binary_cross_entropy_with_logits(input=output,target=target)

In [6]:

#Create a federated model using the build model function and dataset
fl_model = FederatedModel(build_model=net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with fledge. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [7]:
collaborator_models = fl_model.setup(num_collaborators=2)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

TypeError: 'Adam' object is not callable

In [ ]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

In [ ]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,{'rounds_to_train':10})

In [ ]:
#Save final model
final_fl_model.save('final_model')